<a href="https://colab.research.google.com/github/li-chu-hao/bioinfo/blob/main/from_RNA_to_protein.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bioinformatics experiment: From RNA to protein

## Introduction

问题：如何把片段化的碱基序列还原为蛋白质？

**实验目的：**

- 通过完成核苷酸序列的拼接、翻译、折叠，对生物信息学有进一步的认识。
- 认识当前最主要的生物数据：序列数据
- 了解如何使用计算机处理序列数据。
- 了解人工智能在生物信息中的应用。

**实验步骤：**

0. 运行你的第一个python程序。
1. 对RNA进行模拟测序，生成核苷酸序列片段。
2. 序列拼接
3. 翻译为氨基酸
4. 氨基酸折叠


## Step 0：运行你的第一个python程序：

In [ ]:
print("\n--------------")
print("Hello world!")
print("Your python environment has been successfully set up! ")
print("--------------")

## Step 1: 对RNA进行模拟测序，生成核苷酸序列片段（reads）。

根据血红蛋白亚基β的mRNA序列，生成长为50个碱基的reads，测序深度为100X。reads的长度和测序深度对序列拼接非常关键，reads太短或者测序深度不足都会导致序列拼接失败。

1.1 下载血红蛋白亚基β的mRNA序列，保存到`rna.fna`文件。

In [ ]:
!wget -O "rna.fna" "https://www.ncbi.nlm.nih.gov/sviewer/viewer.cgi?tool=portal&save=file&log$=seqview&db=nuccore&report=fasta_cds_na&id=1401724401&extrafeat=null&conwithfeat=on&hide-cdd=on&ncbi_phid=CE89C6D35DF019A1000000000AD308E1"

1.2 查看`rna.fna`文件内容

In [ ]:
!cat rna.fna

1.3 对RNA进行模拟测序，生成核苷酸序列片段。

安装依赖库：

In [ ]:
!python3 -m pip install biopython

生成reads：

In [ ]:
# @title 生成reads代码
# 1. 读取序列。
# 2. 根据reads长度、覆盖度生成随机数。
# 3. 把生成的随机n数作为序列的起点，n+read length作为终点，contigs.
import random
from Bio import SeqIO

seq = SeqIO.read("rna.fna", "fasta")

contig_len = 50 # @param {type:"integer"}
#@markdown - 指定生成的read长度。

coverage = 100 # @param {type:"integer"}
#@markdown - 指定测序深度。

print("\n序列长度：", len(seq.seq))
contig_num = int(len(seq.seq)*coverage / contig_len)

print("生成read的长度：", contig_len)
print("生成read的覆盖度（平均每个碱基被测序到的次数）：", coverage)
print("预计生成的reads条数：", contig_num)

start = 0
end = len(seq.seq) - contig_len

with open("01_output_reads.txt", "w") as f:
    start_points = [random.randint(start, end) for i in range(contig_num)]
    for n in start_points:
        print(str(seq.seq[n:n+contig_len]), file=f)

print("\n运行完毕！请检查结果文件'01_output_reads.txt'。")


查看生成的reads文件的前10行：

In [ ]:
!head 01_output_reads.txt

## Step 2: 序列拼接

拼图游戏（简单版20块）：<div style="display:inline-block;text-align:left;width:140px;"><a style="color:#444" href="https://www.jigidi.com/solve/gk0vdroq/teacup-with-jonquils/"><img style="display:block;border:none;margin:5px 0 0 0;" src="//thumbs.jigidi.com/embed.php?t=s_GK0VDROQ_20" alt="Jigsaw puzzle - Teacup With Jonquils"></a></div>

拼图游戏（纯黑25块）：<div style="display:inline-block;text-align:left;width:140px;"><a style="color:#444" href="https://www.jigidi.com/solve/q493fr9l/solid-black/"><img style="display:block;border:none;margin:5px 0 0 0;" src="//thumbs.jigidi.com/embed.php?t=s_Q493FR9L_25" alt="Jigsaw puzzle - solid_black"></a></div>

拼图程序：
1. **取出拼图**：摆放好全部拼图，执行步骤2。
2. **比较两块拼图**：取拼图i，与另一块尚未与i比较过的拼图比较。
3. **根据两块拼图的比较结果决定下一步**：如果匹配成功，则合并在一起，然后执行步骤4；如果匹配失败，则回到步骤2。
4. **根据是否全部拼完决定下一步**：如果已经全部拼好，则执行步骤5，如果没有，则回到步骤2。
5. **展示拼好的拼图**。

序列拼接程序：

序列拼接与拼图是类似的。下面的代码包含了以上五个步骤，但是其顺序被打乱了。请根据每段代码的描述，联系拼图程序中的相应步骤，对代码进行排序，然后运行。

In [ ]:
# 以下为定义函数。不需要修改这部分。
#----------------------------------------------------------
min_overlap = 25 # @param {type:"integer"}
#@markdown - 指定最少有多少个碱基重叠才能合并两条序列。
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio import SeqIO
# “merge_contig”函数的作用是检测两条序列是否有至少“min_overlap”
# 个碱基的重叠，如果是，那就合并到一起。
def merge_contig(a, b, min_overlap=25):
  min_L = min(len(a), len(b))
  # 处理相互包含的
  if a in b:
    return b
  if b in a:
    return a
  # 处理头尾相接的
  # a head overlap with b tail
  for i in range(min_overlap, min_L+1):
    if a[:i] == b[-i:]:
      c = b + a[i:]
      return c
  # a tail overlap with b head
  for i in range(min_overlap, min_L+1):
    if b[:i] == a[-i:]:
      c = a + b[i:]
      return c
  return None
#----------------------------------------------------------

##############################################
# 请阅读代码的描述，修改下面五个片段的顺序。#
##############################################

# E. 读取之前生成的reads。
#----------------------------------------------------------
reads = open("01_output_reads.txt").read().splitlines()
#----------------------------------------------------------

# D. 把拼接好的序列保存为文件。
#----------------------------------------------------------
with open("02_OLC_assembly.txt", "w") as f:
    record = SeqRecord(
        Seq(b[0]),
        id="HBB_assembly",
        description="HBB assembly by OLC",
)
    SeqIO.write(record, f, "fasta")
print("\n运行完毕！请检查'02_OLC_assembly.txt'文件。")
#----------------------------------------------------------

# B. 如果b列表只剩下一条reads，则说明已经拼接完整，跳出循环。
#   如果还有多条reads，则把a、b列表互换，重复前面一步
#----------------------------------------------------------
  if len(b) == 1:
    break
  else:
    a, b = b, a
#----------------------------------------------------------

# A. 每次从a取一条reads，尝试和b中的reads拼接，如果能拼上，
#   则合并为一条reads，如果拼不上，则加入b中，直到a为空。
#----------------------------------------------------------
while True:
  while len(a) != 0:
    a_read = a.pop()
    for i in range(len(b)):
      b_read = b[i]
      c = merge_contig(a_read, b_read, min_overlap=min_overlap)
      if c:
        b[i] = c
        break
    else:
      b.append(a_read)
#----------------------------------------------------------

# C. 定义a、b两个空列表。把reads放入a中。
#----------------------------------------------------------
a = reads
b = []
#----------------------------------------------------------

In [ ]:
# 运行这里查看参考答案
answer = [69, 67, 65, 66, 68]
print([chr(i) for i in answer])

检查拼接是否正确：

In [ ]:
print("拼接结果：\n")
!cat 02_OLC_assembly.txt

print()
ref = SeqIO.read("rna.fna", "fasta")
assembly = SeqIO.read("02_OLC_assembly.txt", "fasta")
if ref.seq == assembly.seq:
  print("恭喜！拼接正确！")
else:
  print("拼接结果与参考序列不一致！")

探索：

- 不同的reads长度、测序深度对拼接结果有什么影响？
- 如果min_overlap参数设置为1，拼接出来的序列是怎样的？

## Step 3: 翻译为氨基酸

![密码子表](https://cdn1.byjus.com/wp-content/uploads/2017/11/Genetic-code-table-1.png)

In [ ]:
from Bio import SeqIO
seq = SeqIO.read("02_OLC_assembly.txt", "fasta")
seq.seq = seq.seq.translate()
seq.id = "HBB_amino_sequence"
seq.description = "HBB amino sequence"
with open("translated.txt", "w") as f:
    SeqIO.write(seq, f, "fasta")

print("RNA序列翻译为氨基酸序列，运行完毕！")

In [ ]:
!cat translated.txt

## Step 4: 氨基酸折叠

4.1 运行alphafold

打开上一步生成的`translated.txt`文件，把其中的氨基酸序列复制到<a href="https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb" target="_blank">colabfold网站</a>中的`query_sequence`处（不需要复制最末端的`*`号，该符号代表终止密码子），然后点击上方菜单栏中的“代码执行程序，然后点击“全部运行”。模拟折叠该氨基酸序列大概需要5-7分钟。

4.2 查看pdb文件

先下载示例结果文件：[AF-P68871-F1-model_v4.pdb](https://alphafold.ebi.ac.uk/files/AF-P68871-F1-model_v4.pdb)，然后打开[www.rcsb.org/3d-view](https://www.rcsb.org/3d-view)，把下载好的pdb文件上传到rcsb网站，查看其蛋白结构。

4.3 查看alphafold结果

运行完毕后，会自动下载一个压缩包，打开该压缩包，可以看到5个以".pdb"结尾的文件，分别对应5种折叠方案。把第一个pdb文件提交到[www.rcsb.org/3d-view](https://www.rcsb.org/3d-view)，查看蛋白质的三维结构。



4.3 比较蛋白结构

进入[www.rcsb.org/alignment](https://www.rcsb.org/alignment)，把参考的pdb文件和刚刚用alphafold生成的pdb文件进行比较。